In [1]:
! pip install pennylane-lightning[gpu]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.6 MB/s eta 0:00:00


In [2]:
import pennylane as qml
import pennylane.numpy as np
from pennylane.templates import ApproxTimeEvolution

In [3]:
dev = qml.device("default.qubit", wires=9)
# 0: ancillary, 1 ~ 14: LUMO+3 α β, LUMO+2 α β, ... , HOMO-2 α β

In [18]:
def Prep():
    qml.CNOT(wires=[0, 5])
    qml.CNOT(wires=[0, 6])
    angle = - 2 * np.arccos(np.sqrt(1/2))
    qml.CRY(angle, wires=[0, 4])
    qml.CNOT(wires=[4, 3])
    qml.CNOT(wires=[4, 5])
    qml.CNOT(wires=[4, 6])

    qml.CNOT(wires=[0, 7])
    qml.CNOT(wires=[0, 8])
    qml.CRY(angle, wires=[0, 2])
    qml.CNOT(wires=[2, 1])
    qml.CNOT(wires=[2, 7])
    qml.CNOT(wires=[2, 8])

def Prep_inv():
    qml.CNOT(wires=[2, 8])
    qml.CNOT(wires=[2, 7])
    qml.CNOT(wires=[2, 1])
    angle = - 2 * np.arccos(np.sqrt(1/2))
    qml.CRY(-angle, wires=[0, 2])
    qml.CNOT(wires=[0, 8])
    qml.CNOT(wires=[0, 7])

    qml.CNOT(wires=[4, 6])
    qml.CNOT(wires=[4, 5])
    qml.CNOT(wires=[4, 3])
    qml.CRY(-angle, wires=[0, 4])
    qml.CNOT(wires=[0, 6])
    qml.CNOT(wires=[0, 5])

In [23]:
def make_hamiltonian(distance):
    symbols, coordinates = (['Be', 'H', 'H'], np.array([0., 0., 0., 0., 0., -distance, 0., 0., distance]))
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, 
                                                mapping='jordan_wigner',#'bravyi_kitaev', 
                                                wires=[8 - i for i in range(8)],
                                                active_electrons=4,
                                                active_orbitals=4)
                                                #basis='6-31g')
    print(qubits)
    return H

In [24]:
@qml.qnode(dev)
def circuit(distance, epsilon, time=0.1, trotterN=5):
    qml.Hadamard(wires=0)
    Prep()
    H = make_hamiltonian(distance)
    ApproxTimeEvolution(H, time, trotterN)
    Prep_inv()
    qml.PhaseShift(epsilon * time, wires=0)
    qml.Hadamard(wires=0)
    return qml.probs(wires=0)

In [39]:
print(circuit(1.3, -15.8, time=0.1))

8
[0.70077985 0.29922015]
